*   **Año:** 2024
*   **Alumno/a:** Alexey Marassi
*   **Legajo:** 19070876

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [1]:
import pydantic

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [3]:
from pydantic import BaseModel
class Client(BaseModel):
  dni: int
  nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [4]:
usuario = Client(dni=39755010, nacionalidad='Argentina')
print(usuario)

dni=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [11]:
nuevo_usuario = Client(dni='12412', nacionalidad='Argentina')
#nuevo_usuario = Client(dni='hola', nacionalidad='Argentina')

# falla solo en el caso cuando el dni es exlicitamente un str que tiene letras
# si el str consiste de numeros hace la convercion a int

print(nuevo_usuario)

dni=12412 nacionalidad='Argentina'


Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [15]:
from pydantic import BaseModel, validator, ValidationError
from datetime import datetime # para usar datetime
# nacionalidad inexistente
# dni negativos
# poder agregar la fecha del registro < fecha de hoy


lista_nacionalidades = ['Argentina',
                        'Chile',
                        'Uruguay',
                        'Paraguay',
                        'Bolivia',
                        'Peru',
                        'Colombia',
                        'Venezuela',
                        'Ecuador',
                        'Guyana',
                        'Suriname',
                        'Francia']

class Client(BaseModel):
  dni: int
  nacionalidad: str
  fecha_registro: datetime

  @validator('nacionalidad')
  def validar_nacionalidad(cls, valor): #se usa 'cls' en vez de 'self' en pydantic
    if valor not in lista_nacionalidades:
      raise ValueError('Nacionalidad no esta en la lista de nacionalidades')
    return valor

  @validator('dni')
  def validar_dni(cls, valor):
    if valor < 0:
      raise ValueError('DNI no puede ser negativo')
    return valor

  @validator('fecha_registro')
  def validar_fecha_registro(cls, valor):
    if valor > datetime.now():
      raise ValueError('Fecha de registro no puede ser en el futuro')
    return valor

<ipython-input-15-acc580f1a839>:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('nacionalidad')
<ipython-input-15-acc580f1a839>:32: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('dni')
<ipython-input-15-acc580f1a839>:38: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more 

In [16]:
# nacionalidad que no esta en la lista
usuario = Client(dni=39755010, nacionalidad='Mexico', fecha_registro=datetime.now())

ValidationError: 1 validation error for Client
nacionalidad
  Value error, Nacionalidad no esta en la lista de nacionalidades [type=value_error, input_value='Mexico', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [18]:
# dni negativo
usuario = Client(dni=-39755010, nacionalidad='Argentina', fecha_registro=datetime.now())

ValidationError: 1 validation error for Client
dni
  Value error, DNI no puede ser negativo [type=value_error, input_value=-39755010, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [21]:
# nacionalidad que no esta en la lista
usuario = Client(dni=39755010, nacionalidad='Argentina', fecha_registro='2030-01-05')

ValidationError: 1 validation error for Client
fecha_registro
  Value error, Fecha de registro no puede ser en el futuro [type=value_error, input_value='2030-01-05', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [23]:
!pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.6 MB/s eta 0:00:00


In [44]:
import pandera as pa
import pandas as pd

In [45]:
from pandera import Check
from datetime import datetime

cliente =  pa.DataFrameSchema(
    {
        "dni": pa.Column(int, Check.greater_than(0)),
        "nacionalidad": pa.Column(str, Check.isin(lista_nacionalidades)),
        "fecha_registro": pa.Column(datetime, Check.less_than_or_equal_to(datetime.now()))
    }
)

In [52]:
data = {
    "dni": [19070876, 20567434, -123, 1, 56756],
    "nacionalidad": ['Argentina', 'Chile', 'Brazil', 'Alemania', 'Colombia'],
    "fecha_registro": [datetime(1998,1,5),datetime(2000,2,10),datetime(1996,3,1),datetime(1000,10,1),datetime(2300,2,2)]
}

df = pd.DataFrame(data)

# funcion para validar cada registro del df
def validar_registro(registro):
    try:
        cliente.validate(pd.DataFrame([registro])) #validar por fila
        print("Registro valido:\n", registro)
        print("----------------------------------------------------")
    except pa.errors.SchemaError as e:
        print("Registro invalido:\n", registro)
        print("----------------------------------------------------")

# iterar sobre las filas del df y validar cada una
for index, row in df.iterrows():
    validar_registro(row)

Registro valido:
 dni                          19070876
nacionalidad                Argentina
fecha_registro    1998-01-05 00:00:00
Name: 0, dtype: object
----------------------------------------------------
Registro valido:
 dni                          20567434
nacionalidad                    Chile
fecha_registro    2000-02-10 00:00:00
Name: 1, dtype: object
----------------------------------------------------
Registro invalido:
 dni                              -123
nacionalidad                   Brazil
fecha_registro    1996-03-01 00:00:00
Name: 2, dtype: object
----------------------------------------------------
Registro invalido:
 dni                                 1
nacionalidad                 Alemania
fecha_registro    1000-10-01 00:00:00
Name: 3, dtype: object
----------------------------------------------------
Registro invalido:
 dni                             56756
nacionalidad                 Colombia
fecha_registro    2300-02-02 00:00:00
Name: 4, dtype: object
-------